# Selenium 국회의원 데이터 스크래핑

## Selenium 및 웹 드라이버 설치

In [ ]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 4.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.3 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates In

## 라이브러리 import

In [ ]:
import os
import shutil
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from urllib.request import urlretrieve 

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-div-shm-usage')

# 국회의원 기본정보 스크래핑

In [ ]:
def get_politician_info(dir_name, wd):
  
  profile_info = wd.find_element_by_class_name('depAnalProfileTbl')

  cols = profile_info.find_elements_by_tag_name('th')
  columns = [col.text for col in cols]

  infos = profile_info.find_elements_by_tag_name('td')
  detail_info = [info.text for info in infos]

  info_dic = { col:info for col, info in zip(columns, detail_info) }

  return info_dic

# 국회의원 뉴스 인용문

In [ ]:
def get_news_quote(dir_name, name, wd, news_max=5):

  page_no = 0
  news_count = 0
  news_total = wd.find_element_by_id('newsInQoutTotalCount').text[1:-2]
  news_file = open(dir_name + '/' + name + '뉴스 인용문.txt', 'w') #쓰기 모드 파일 생성

  while True:
    try:
      page_no += 1
      paging = wd.find_element_by_id('newsInQuotListPaging')
      pagination = paging.find_element_by_class_name('pagination')
      pagination.find_element_by_link_text(str(page_no)).click()
      time.sleep(1)
      print(f"뉴스 인용문 {page_no} 페이지")
      
      box_list = wd.find_element_by_id('newsInQuotList')
      quotes = box_list.find_elements_by_tag_name('b')
      for quote in quotes:
        news_file.write(quote.text + '\n') #파일 쓰기
        news_count += 1
        print(f" {news_count}/{news_total} ", [quote.text])

        if news_count >= news_max:
          break

      if news_count >= news_max:
          break

      next_button = pagination.find_element_by_tag_name('li')[-1]
      if next_button.get_attribute('class') == 'disabled':
        break

      if page_no % 5 == 0:
        next_button.find_element_by_tag_name('a').click()
        time.sleep(1)

    except:
      break

  news_file.close()

## 국회의원 회의록

In [ ]:
def get_record_quote(dir_name, name, wd, record_max=5):

  page_no = 0
  record_count = 0
  record_total = wd.find_element_by_id('recordTotalCount').text
  record_file = open(dir_name + '/' + name + '회의록 인용문.txt', 'w') #쓰기 모드 파일 생성

  while True:
    try:
      page_no += 1
      paging = wd.find_element_by_id('recordTableListPaging')
      pagination = paging.find_element_by_class_name('pagination')
      pagination.find_element_by_link_text(str(page_no)).click()
      time.sleep(1)
      print(f"회의록 인용문 {page_no} 페이지")

      record_table = wd.find_element_by_id('recordTableList')
      record_list = record_table.find_elements_by_tag_name('a')
      for record in record_list:
        record.click()
        time.sleep(1)

        quote_list = wd.find_element_by_id('birefTableList')
        quotes = quote_list.find_elements_by_tag_name('td')
        for quote in quotes:
          record_file.write(quote.text + '\n') #파일 쓰기

        record_count += 1
        print(f" {record_count}/{record_total} ", [quote.text])

        if record_count >= record_max:
          break

      if record_count >= record_max:
          break

      next_button = pagination.find_element_by_tag_name('li')[-1]
      if next_button.get_attribute('class') == 'disabled':
        break

      if page_no % 5 == 0:
        next_button.find_element_by_tag_name('a').click()
        time.sleep(1)

    except:
      break

  record_file.close()

## `scraping()`: 스크래핑

In [ ]:
def scraping(dir_name, politician_max=3):
  page_no = 0
  politician_count = 0
  politician_df = pd.DataFrame()

  wd = webdriver.Chrome('chromedriver', options=chrome_options)
  wd.execute_script('window.open("about:blank", "_blank");') #새 창으로 열기
  tabs = wd.window_handles #새 창을 탭으로 열기

  while True:
    try:
      page_no += 1
      wd.switch_to.window(tabs[0]) #첫 번째 탭에 접근

      url = f"https://www.bigkinds.or.kr/v2/depthAnalysis/assembly.do?page={page_no}"
      wd.get(url)
      politician_total = wd.find_element_by_xpath('//*[@id="contents"]/div/div[3]/div/button[1]/span').text[1:-1]
      politician_items = wd.find_elements_by_class_name('assembly-item')

      if not politician_items:
        break
      
      for item in politician_items:
        wd.switch_to.window(tabs[0])

        politician_count += 1
        print("------------------------------------------------------")
        print(f"국회의원 {politician_count}/{politician_total} 명")
        name = item.find_element_by_class_name('assembly-item__profile__name').text
        name = name.split('\n')[0]
        print(name)

        deatil_link = item.find_element_by_tag_name('a').get_attribute('href')
        wd.switch_to.window(tabs[1]) #두번째 탭에 로드
        wd.get(deatil_link)

        print("[프로필 이미지 다운로드]")
        profile_image = wd.find_element_by_class_name('depAnalProfileImg')
        image_src = profile_image.find_element_by_tag_name('img').get_attribute('src')
        file_name = dir_name + '/' + name + '.jpg'
        print(file_name)
        urlretrieve(image_src, file_name) #(소스, 파일명)

        print("[기본 정보 스크래핑]")
        politician_dic = get_politician_info(dir_name, wd) #함수 호출
        print("  ", politician_dic)
        politician_df = politician_df.append(politician_dic, ignore_index=True)

        print("[뉴스 인용문 스크래핑]")
        wd.find_element_by_id('depthAnalTab2_1').find_element_by_tag_name('a').click()
        get_news_quote(dir_name, name, wd) #함수 호출

        print("[회의록 인용문 스크래핑]")
        get_record_quote(dir_name, name, wd) #함수 호출

        if politician_count >= politician_max:
          break
        
      if politician_count >= politician_max:
          break
    
    except AttributeError as e:
      print(e)
      break

    except NoSuchElementException as e:
      print(e)
      break

  wd.close()

  return politician_df

# 스크래핑 시작

In [ ]:
dir_name = './politician'
if os.path.isdir(dir_name):
  shutil.rmtree(dir_name)

os.makedirs(dir_name)
print(f"{dir_name} 디렉토리 생성")

politician_df = scraping(dir_name)

./politician 디렉토리 생성
------------------------------------------------------
국회의원 1/301 명
강기윤 (姜起潤)
[프로필 이미지 다운로드]
./politician/강기윤 (姜起潤).jpg
[기본 정보 스크래핑]
   {'이름': '강기윤(姜起潤)', '이름(영문)': 'KANG GIYUN', '출생일': '1960-06-04', '정당': '미래통합당', '지역구': '경남 창원시성산구', '소속위원회': '보건복지위원회', '당선기록': '재선(19대, 21대)', '사무실전화': '02-784-1751', '홈페이지': 'http://blog.naver.com/ggotop', '이메일': 'ggotop@naver.com', '보좌관': '강종길 , 김홍광', '경력': '[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사 창원대학교 대학원 행정학 박사 [경력] 보건복지위원회 위원 미래통합당 소상공인살리기 특별위원회 부위원장 미래통합당 경남도당 민생특위 위원장 제19대 국회의원 (새누리당/경남 창원시 성산구) 새누리당 원내부대표'}
[뉴스 인용문 스크래핑]
뉴스 인용문 1 페이지
 1/54  ['문재인 케어로 남발했던 건강보험기금이 고갈돼 정부가 건강보험료를 인상하려 하고 있는데, 경제적 위기로 국민들이 고통받고 있는 상황에서 시기가 적절하지 않다']
 2/54  ['잠복기가 5일 정도라는데, 광화문 집회 직후 확진자가 발생했으면 10일 이전에 코로나19 감염이 됐다는 의미 아닌가 그것을 마냥 광화문 집회가 확진자를 양산하는 주범이라는 쪽으로 혹세무민해서는 안 된다']
 3/54  ['의협이 지금 당장 업무에 복귀해서 모든 국민이 걱정하는 코로나 방역에 최선을 다해줄 것을 간곡히 요청한다 지금 당장 보건복지부 차관이 의협에 지금은 일단 코로나 방역에 힘을 쏟자는 메시지를 당장 내달라']
 4/54  ['아나볼릭 스테로이드의 부작용의 위험성과 현재 오남용이 심각한 

## 스크래핑 확인

In [ ]:
politician_df

,경력,당선기록,보좌관,사무실전화,소속위원회,이름,이름(영문),이메일,정당,지역구,출생일,홈페이지
0,[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사...,"재선(19대, 21대)","강종길 , 김홍광",02-784-1751,보건복지위원회,강기윤(姜起潤),KANG GIYUN,ggotop@naver.com,미래통합당,경남 창원시성산구,1960-06-04,http://blog.naver.com/ggotop
1,,초선(21대),"박홍규 , 정운태",,국방위원회,강대식(姜大植),KANG DAESIK,,미래통합당,대구 동구을,1959-11-02,
2,2018. 10. ~ 2019. 5. 민주연구원 자치발전연구센터 본부장 2016. ...,초선(21대),유진우,02-784-2747~9,교육위원회,강득구(姜得求),KANG DEUKGU,mainsail440@daum.net,더불어민주당,경기 안양시만안구,1963-05-27,https://blog.naver.com/dulipapa


In [ ]:
!ls politician/

'강대식 (姜大植).jpg'		    '강득구 (姜得求)회의록 인용문.txt'
'강대식 (姜大植)뉴스 인용문.txt'    '강기윤 (姜起潤).jpg'
'강대식 (姜大植)회의록 인용문.txt'  '강기윤 (姜起潤)뉴스 인용문.txt'
'강득구 (姜得求).jpg'		    '강기윤 (姜起潤)회의록 인용문.txt'
'강득구 (姜得求)뉴스 인용문.txt'


In [ ]:
from google.colab import files
files.download('./politician/강대식 (姜大植).jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cat './politician/강대식 (姜大植)뉴스 인용문.txt'

금호강을 횡단하는 이시아폴리스 진입도로 교량 건설이 예정돼 있는데 엑스코선 교량을 따로 짓지 말고 여기에 함께 놓으면 경제성이 대폭 향상된다 또 이시아폴리스 인근 연경지구 개발로 이용수요가 더욱 증가할 것
통합신공항 이전 16단계 중 9단계가 끝났다. 제일 우려한 부분이 11단계로 '사업자 선정' 문제다. 사업자가 빨리 선정되면 사업이 순조로울 것
공기업이든 민간기업이든 컨소시엄 형태가 될 텐데 중요한 점은 어느 쪽이 됐든 누가 이득 없는 사업을 맡겠느냐 것이다. 경제성이 있어야 투자를 할 것 현행법으로는 민간에서 투자가 이루어질지 우려스러운 부분이 있어 유인책이든 무엇이든 방안을 모색해야 할 것
자칫하면 대형참사가 났을 수 있지만, 양측 인명 피해가 없어 천만다행한 일
자칫하면 대형참사가 났을 수 있지만, 양측 인명 피해가 없어 천만다행한 일 특히, 잠수함의 경우 부상상태 항해시 충돌에 매우 취약하기 때문에 더더욱 사고예방을 위해 배와 교신, 항로 이탈시 기적소리 발송, 안전속력 준수, 충돌회피를 위한 동작 등을 철저히 해야 한다 해군은 충돌사고시 큰 인명피해가 있을 뿐 아니라 전력공백이 발생하기 때문에 해상수칙만 지..


In [ ]:
!cat './politician/강대식 (姜大植)회의록 인용문.txt'


그렇지 않지요? 그런데 7월 6일 자에 보면 정세현 민주평통자문회의 수석부의장께서 김어준의 뉴스공장에 출연해 가지고 9․19 군사합의 이후 실시한 한미연합훈련에 대해서 군사합의 위반이라고 이렇게 주장을 했는데 여기에 대해서 장관께서 어떻게 생각하십니까?

허위사실에 대해서 이야기를 할 때에는 국방부에서의 명확한 답변을 내놔야 된다고 저는 그렇게 보고 있거든요. 우리 국민들이 봤을 때 호도되지 않고 정말 정확한 팩트 있는 그런 사실들을 우리 국민이 알 수 있게끔 이렇게 국방부에서 해 줘야 된다고 생각하는데 장관님 어떻게 생각하세요?

수석부의장은 대통령의 어떤 명을 받아서 이렇게 하는 아주 중요한 위치에 계시는 분이 이런 허위사실에 대해서…… (발언시간 초과로 마이크 중단) 1분만 더 주십시오.

또 한 가지 물어보겠습니다. 한미연합훈련이 9․19 합의에 위반됩니까?

예, 됐습니다. 됐습니다.

예, 무슨 말씀인지 알겠습니다. 이게 꼭히 통합신공항이 510만 대구․경북민들의 염원이 담긴 사업이다 이렇게는 제가 말씀을안 드리겠습니다. 왜냐하면 생각하기도 싫지만이 사업이 만약에 좌초된다고 봤을 때는 대구․경북을 넘어서서 예측하기 힘든 정치적 갈등의소용돌이 속에 빠질 수 있다 이런 생각을 한번해 보고요. 주무부처로서 책임감을 가지고 며칠 남지는 않았지만 그동안 수십 년간 소음 피해로 인해서 또고도 제한으로 인해서 재산권 침해로 인해서 고통받은 대구시민들의 희생을 헤아려서 마지막까지 군위군을 설득하는 데 최선을 다해 줄 수 있겠습니까?

그런데 아직까지 군위군은 그럴 마음이 전연 없고요. 언론에 보면 장관님이 내일 일정으로 군위군수하고 면담이 있는 걸로 알고 있는데 혹시 설득을 할 수 있는 대안을 갖고 계십니까?

장관님, 조금 전에 질의했던 내용과 연결하겠습니다. 군위군이 진짜로 유치하면 제일 클리어합니다, 그렇지요?

뭐 조금만이라도 그런……

방위력개선비 중 약 한 9200억 원이 삭감된 이후에 2021년 예산편성에서 감액된 사업 예산을 재배정받고 정상적인 사